# Experimentos

In [1]:
import torch


In [2]:
def print_gpu_memory(prefix=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"{prefix} Memory Allocated: {allocated:.2f} MB")
        print(f"{prefix} Memory Reserved: {reserved:.2f} MB")
    else:
        print("CUDA is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 

print_gpu_memory()

 Memory Allocated: 0.00 MB
 Memory Reserved: 0.00 MB


In [3]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))

import src.models.unets as unets
import src.data.preprocess_data as data
import src.training.train_model as train
import src.models.hrnets as hrnets

from torch.utils.data import DataLoader
import torch.optim as optim



/home/jonathan/UrbanizedAreasSegmentation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

tiles = ['032027']#, '032026'] 
num_subtiles = 6
classes_mode = 'binary'

if classes_mode == 'type':
    num_classes = 5
elif classes_mode == 'density':
    num_classes = 4
elif classes_mode == 'binary':
    num_classes = 2
else:
    num_classes = 9




In [5]:
model_param_grid = {
    #model params:
    'model' : [f'UNet-256-{classes_mode}', #ok
               f'UNet-64-{classes_mode}', #ok
               f'UNetResNet34-224-{classes_mode}', #ok
               f'UNetEfficientNetB0-224-{classes_mode}', 
               f'UNetConvNext-224-{classes_mode}',
               f'HRNet_w18-512-{classes_mode}',
               f'HRNet_w32-512-{classes_mode}',
               f'HRNet_w48-512-{classes_mode}'
               ],
        
    #training params
        # loss
    'loss': ['BCE-dice'],#'BCE', 'dice', 'BCE-dice'],
    'weighted_loss': [False],
    'learning_rate': [0.0001],    
    'epochs' : [10],
    'patience' : [3],
    'batch_size' : [4]
       
}


In [6]:

train_files, val_files, test_files = data.train_val_test_stratify(tiles, 
                                                                  num_subtiles,
                                                                    train_size = 0.7, 
                                                                    val_size = 0.15, 
                                                                    stratify_by = classes_mode)



Training set size: 25
Validation set size: 5
Test set size: 6
saved /home/jonathan/UrbanizedAreasSegmentation/config/train_val_test_split-6_subtiles-032027-stratified_by_binary.yaml


In [7]:
for model_params in train.iterate_parameter_grid(model_param_grid):

    #definitions
    model_name = model_params['model'] 
    patch_size = int(model_name.split('-')[1])
    print(model_params)
    model_class = model_name.split('-')[0]

    batch_size = model_params['batch_size'] 


    #load data
    train_dataset = data.SubtileDataset(train_files, 
                    num_subtiles = num_subtiles, 
                    classes_mode = classes_mode, 
                    patch_size=(patch_size, patch_size), 
                    stride=patch_size//2, 
                    data_augmentation = True, 
                    return_imgidx = False,
                    return_nans=False,
                    debug=False)

    val_dataset = data.SubtileDataset(val_files, 
                        num_subtiles = num_subtiles, 
                        classes_mode = classes_mode, 
                        patch_size=(patch_size, patch_size), 
                        stride=patch_size//2,
                        data_augmentation = False, 
                        return_imgidx = False,
                        return_nans=False,
                        debug=False)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    if model_name.startswith('UNet-'):
        model = unets.UNet(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetResNet34-'):
        model = unets.UNetResNet34(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetEfficientNetB0-'):
        model = unets.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetConvNext-'):
        model = unets.UNetConvNext (in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('HRNet_w18'):
        model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w18", pretrained=True, debug=True).to(device)
    elif model_name.startswith('HRNet_w18'):
        model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w32", pretrained=True, debug=True).to(device)
    elif model_name.startswith('HRNet_w18'):
        model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w48", pretrained=True, debug=True).to(device)
        

    optimizer = optim.Adam(model.parameters(), lr=model_params['learning_rate'], weight_decay=1e-5)
    train.train_model(model, 
                        train_loader, 
                        val_loader, 
                        epochs=model_params['epochs'], 
                        loss_mode = model_params['loss'],
                        optimizer = optimizer, 
                        device = device,
                        num_classes = num_classes, 
                        patience = model_params['patience'],
                        label_counts = None,
                        show_batches = 1, 
                        save_to = model_name+'.pth')


    


{'model': 'UNet-256-binary', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3, 'batch_size': 4}
Doing data augmentation...
Augmented 0 images, of 4900
Model is already saved
Loading it e continue training...
LOSS mode: BCE-dice
Training for epoch 5 until epoch 10
Training Epoch 5/10
Running an epoch on the train mode.


100%|██████████| 1225/1225 [04:23<00:00,  4.64it/s]


Train Loss: 0.22041805444931498
Train Accuracy: 0.9858588576316833
              precision    recall  f1-score    support
Class 0        0.992807  0.992247  0.992527  303913056
Class 1        0.864461  0.873073  0.868746   17213076
weighted avg   0.985927  0.985859  0.985892  321126144
macro avg      0.928634  0.932660  0.930636  321126144
micro avg      0.985859  0.985859  0.985859  321126144
Running an epoch on the validation mode.


  9%|▉         | 23/245 [00:02<00:19, 11.53it/s]

Error while showing sample images...
name 'view' is not defined


...

Error while showing sample images...
name 'view' is not defined


 58%|█████▊    | 36/62 [00:05<00:03,  6.65it/s]

Error while showing sample images...
name 'view' is not defined


100%|██████████| 62/62 [00:09<00:00,  6.62it/s]

Train Loss: 0.37467716689012487
Train Accuracy: 0.9402394890785217
              precision    recall  f1-score   support
Class 0        0.940239  1.000000  0.969199  60387136
Class 1        0.000000  0.000000  0.000000   3838135
weighted avg   0.884050  0.940239  0.911280  64225272
macro avg      0.470120  0.500000  0.484600  64225272
micro avg      0.940239  0.940239  0.940239  64225272


In [8]:
ims = []
for f in originals:
    with rasterio.open(f) as im:        
        #window = rasterio.windows.Window(x+subtile_x ,y+subtile_y , patch_size, patch_size) #TODO mudar esse hardcoded
        im = im.read()#window = window)
        im = im[1:4,:,:]
        im = im.transpose((1, 2, 0))
        im = np.clip(im, 0, 10000)
        im = im-np.min(im)
        im = im/np.max(im)
        ims.append(im)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

limx = (1000,1700)
limy = (1000,1700)
for im, mask in zip(ims, results):
    plt.figure(figsize=(10,10))
    plt.imshow(im[limx[0]:limx[1],limy[0]:limy[1],:])
    for c in range(5):
        plt.figure(figsize=(10,10))
        plt.imshow(mask[c,limx[0]:limx[1],limy[0]:limy[1]], cmap='gray')
    print(np.min(mask), np.max(mask), mask.shape)
    #plt.figure(figsize=(10,10))
    #plt.imshow(mask>0.5)




NameError: name 'originals' is not defined

In [ ]:



import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def plot_satellite_with_mask(ax, satellite_image, mask, title="Satellite Image with Mask Overlay",
                             mask_color=(1, 0, 0), mask_alpha=0.5, border_color='red',
                             border_width=2):

    if satellite_image.shape[-1]!=3:
        satellite_image = satellite_image[1:4,:,:]
        satellite_image = satellite_image.transpose((1, 2, 0))
    # Ensure mask is binary
    mask = (mask > 0).astype(int)

    # Create a custom colormap for the mask
    colors = [(0, 0, 0, 0), mask_color + (mask_alpha,)]
    cmap = LinearSegmentedColormap.from_list("custom", colors, N=2)

    # Plot the satellite image
    ax.imshow(satellite_image)
    print(np.min(satellite_image))
    print(np.max(satellite_image))
    # Plot the mask overlay
    #mask_plot = ax.imshow(mask, cmap=cmap)

    # Plot the mask border
    #ax.contour(mask, colors=border_color, linewidths=border_width, levels=[0.5])

    # Set title and remove axis ticks
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    return ax
    return mask_plot

def plot_multiple_satellite_images(satellite_images, masks, titles=None, figsize=(20, 20),
                                   mask_color=(1, 0, 0), mask_alpha=0.5, border_color='red',
                                   border_width=2):
    """
    Plot multiple satellite images with their corresponding masks in subplots.

    Parameters:
    satellite_images (list): List of satellite images (each a 3D numpy array)
    masks (list): List of corresponding masks (each a 2D numpy array)
    titles (list): List of titles for each subplot (optional)
    figsize (tuple): Figure size in inches (width, height)
    mask_color (tuple): RGB color for the mask (values between 0 and 1)
    mask_alpha (float): Alpha value for mask transparency (between 0 and 1)
    border_color (str or tuple): Color of the mask border
    border_width (float): Width of the mask border

    Returns:
    matplotlib.figure.Figure: The created figure
    """
    n = len(satellite_images)
    rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if n == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    for i, (image, mask) in enumerate(zip(satellite_images, masks)):
        if i < n:
            title = titles[i] if titles and i < len(titles) else f"Image {i+1}"
            mask_plot = plot_satellite_with_mask(axes[i], image, mask, title, 
                                                 mask_color, mask_alpha, 
                                                 border_color, border_width)
        else:
            axes[i].axis('off')

    # Add a colorbar to the last subplot
    cbar = fig.colorbar(mask_plot, ax=axes[-1], ticks=[0, 1])
    cbar.set_label('Mask Value')

    plt.tight_layout()
    return fig




In [ ]:
patches com 10% dos dados?
data aug so nas minoritarias
mostrar os erros
olhar o recall
salvar as mascaras para abrir no qgis.
aplicar só no setor censitario?



qual a saida do modelo? probabilidade de cada classes?
qual data augmentation vc fez?
qual as imagens vc ta usando? sentinel? resolucao?
como é a balanced accuracy?

0
loteamento vazio
outros equipamentos
vazio interubano
area urbanizada
